In [1]:
import numpy as np
from numpy import random as rnd
from matplotlib import pyplot as plt
import warnings,os,time,datetime,sys

import torch
import pandas as pd

import gym

In [24]:
class Net(torch.nn.Module):
    def __init__(self,env):
        super(Net,self).__init__()
        self.l0 = torch.nn.Linear(env.observation_space.shape[0],16)
        self.l1 = torch.nn.Linear(16,env.action_space.n)
    def forward(self,x):
        x = torch.nn.functional.relu(self.l0(x))
        x = torch.nn.functional.softmax(self.l1(x))
        return x

In [25]:
def discount_rewards(rewards,gamma=0.99):
    r = np.array([gamma**k*rewards[k] for k in range(len(rewards))])
    r = r[::-1].cumsum()[::-1]
    return r-r.mean()

In [28]:
total_rewards = []

batch_rewards = []
batch_actions = []
batch_states = []

batch_counter = 1

In [47]:
env = gym.make('CartPole-v0')
action_space = np.arange(env.action_space.n)

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


In [30]:
net = Net(env)
optimizer = torch.optim.Adam(net.parameters(),lr=0.01)

In [66]:
s0 = env.reset()

states = []
rewards = []
actions = []

done = False

In [67]:
action_probs = net(torch.FloatTensor(s0)).detach().numpy()
action_c = rnd.choice(action_space,p=action_probs)

C:\Users\Jan\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  


In [68]:
s1,r,done,_ = env.step(action_c)

In [69]:
states.append(s0)
rewards.append(r)
actions.append(action_c)

In [70]:
s0 = s1

In [71]:
total_rewards.append(sum(rewards))

batch_states.extend(states)
batch_rewards.extend(discount_rewards(rewards))
batch_actions.extend(actions)

batch_counter += 1